In [2]:
import pandas as pd
import os 

In [4]:
products_csv = [line_csv for line_csv in os.listdir("data") if "line" in line_csv]
len( products_csv)

77

In [5]:
line_items = pd.concat([pd.read_csv(f"data/{line_csv}") for line_csv in products_csv ])

In [6]:
line_items.shape[0]

61247787

In [9]:
line_items.head(2)

,shipment_id,master_category_id,created_at,price,discount,item_id,quantity,product_id,brand_name,pricer,replaced,cancelled
0,387023,34.0,2018-03-02 17:23:48,64.989998,0.0,3857464,1,14840,Fine Life,Pricer::PerItem,0,0
1,387023,22.0,2018-03-02 17:32:52,22.990000,0.0,3857581,1,66260,Слобода,Pricer::PerItem,0,0


In [35]:
line_items_successful = line_items

In [36]:
line_items_successful.shape[0]

61247787

In [37]:
line_items_successful["total_price_without_dicsount"] = line_items_successful["price"] * (line_items_successful["quantity"] - line_items["replaced"] - line_items["cancelled"])
line_items_successful["total_discount"] = line_items_successful["discount"] * (line_items_successful["quantity"] - line_items["replaced"] - line_items["cancelled"])

In [38]:
line_items_successful[line_items_successful["quantity"] > 1].head(2)

,shipment_id,master_category_id,created_at,price,discount,item_id,quantity,product_id,brand_name,pricer,replaced,cancelled,total_price_without_dicsount,total_discount
5,387023,24.0,2018-03-02 17:38:08,32.990002,0.0,3857716,2,46752,Даниссимо,Pricer::PerItem,0,0,65.980003,0.0
8,387023,24.0,2018-03-02 17:39:22,32.990002,0.0,3857732,2,94317,Даниссимо,Pricer::PerItem,0,0,65.980003,0.0


In [47]:
line_items_successful[(line_items_successful["cancelled"] > 0) & (line_items_successful["discount"]!=0 )].head(2)

,shipment_id,master_category_id,created_at,price,discount,item_id,quantity,product_id,brand_name,pricer,replaced,cancelled,total_price_without_dicsount,total_discount
1021,1236499,144.0,2019-08-21 17:03:14,8.5,5.29,17108481,35,93360,Сады Придонья,Pricer::PerItem,0,1,289.0,179.859999
1022,1236499,144.0,2019-08-21 17:03:21,8.5,6.49,17108488,18,93358,Сады Придонья,Pricer::PerItem,0,1,144.5,110.329996


In [48]:
shipmnet_agg = line_items_successful.groupby("shipment_id").agg(
                                    { "quantity" : "sum",
                                       "item_id" : "count",
                                       "total_price_without_dicsount": "sum", 
                                       "total_discount": "sum",
                                       "cancelled" : "sum",
                                       "replaced" : "sum"
                                      }
                                     )

In [49]:
shipmnet_agg = shipmnet_agg.rename(columns = {"quantity": "total_number_of_products", "item_id": "total_number_of_unique_products"})

In [50]:
shipmnet_agg.head(10)

,total_number_of_products,total_number_of_unique_products,total_price_without_dicsount,total_discount,cancelled,replaced
shipment_id,,,,,,
178163,43,31,7489.499998,320.939995,7,0
273988,113,81,33582.100059,2102.400017,9,4
322307,47,24,3529.400031,1198.849987,1,0
337809,75,45,8686.630035,857.299990,0,2
351762,73,54,8297.440020,831.120012,0,0
354106,50,41,5435.339994,842.840000,2,0
368687,57,43,8112.369992,947.749973,5,2
383088,379,249,56977.640068,1433.969999,8,4
384281,130,123,19397.190060,3057.999992,3,0


In [51]:
shipmnet_agg.to_csv("data/products_per_shipment_aggregation_v2.csv")

In [53]:
shipmnet_agg.shape[0]

2266599